In [1]:
import nibabel as nib
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import random
import matplotlib.pyplot as plt
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from scipy.ndimage import zoom
from glob import glob
from PIL import Image
from tqdm import tqdm
from torch.utils.data import random_split
from torch.utils.data import Subset, DataLoader
from torchsummary import summary
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, confusion_matrix

torch.cuda.manual_seed(42)
np.random.seed(42)
random.seed(42)

if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
torch.backends.cudnn.deterministic = True

## Data Processing and Exploration

### Choosing the right data:

The full dataset has been processed from it's original 350GB+ MRI dataset. The original csv file is filtered to include only 1.5T or 3.0T scans as they contain the most comprehensive sets of MRI scans of differing types (T1-weighted, T2, Bold, etc). 

The clinical dementia rating (cdr) with values (0, 0.5, 1, 2) represent 0 = absent; 0.5 = questionable; 1= present, but mild; 2 = moderate (reference: https://www.sciencedirect.com/topics/neuroscience/clinical-dementia-rating). They have been remapped from (0, 0.5, 1, 2) to (0, 1, 2, 3) due to issues processing it with Torch dataloader, leading to a missing class for 0.5 cdr.

Based on each MRI session value (Label), if there are multiple values for the cdr then the scans are excluded.

In [2]:
diagnostic_file = glob('diagnosis.csv')[0]

diagnostic_df = pd.read_csv(diagnostic_file)

diagnostic_df = diagnostic_df[(diagnostic_df.Scanner == '3.0T') | (diagnostic_df.Scanner == '1.5T')]

multiple_values = diagnostic_df.groupby('Label').filter(lambda group: group['cdr'].nunique() > 1)['Label'].unique()

cdr_map = {0.0: 0, 0.5: 1, 1.0: 2, 2.0: 3}

# Filter out rows with these values in column A
filtered_diagnostic_df = diagnostic_df[~diagnostic_df['Label'].isin(multiple_values)]
filtered_diagnostic_df = filtered_diagnostic_df.drop_duplicates(subset='Label')
# filtered_diagnostic_df['cdr'] = filtered_diagnostic_df['cdr'].map(cdr_map)
filtered_diagnostic_df['file_tag'] = filtered_diagnostic_df.Label.apply(lambda file_name: file_name[-5:])
print(len(filtered_diagnostic_df))

valid_mr_scans_ls = filtered_diagnostic_df.file_tag.to_list()
print(valid_mr_scans_ls[:3])
filtered_diagnostic_df.file_tag

# filtered_diagnostic_df.to_csv('filtered_data.csv')

1379
['d0129', 'd2430', 'd3132']


0       d0129
2       d2430
3       d3132
12      d0371
15      d2340
        ...  
6193    d0148
6194    d2526
6195    d1566
6216    d1717
6217    d0407
Name: file_tag, Length: 1379, dtype: object

In [3]:
data_path = 'bids/' ## Change as needed

def recursive_glob_with_filter(directory, filter_list):
    # Use recursive glob pattern to find all files
    all_files = glob(directory + '/**/*T1*nii.gz', recursive=True)
    
    # Filter files based on the presence of strings from filter_list in the file path
    filtered_files = [file for file in all_files if any(filter_str in file for filter_str in filter_list)]
    
    return filtered_files

file_ls = recursive_glob_with_filter(data_path, valid_mr_scans_ls)
print(file_ls[:2])

['bids\\sub-OAS30001\\ses-d0129\\anat\\sub-OAS30001_ses-d0129_run-01_T1w.nii.gz', 'bids\\sub-OAS30001\\ses-d0129\\anat\\sub-OAS30001_ses-d0129_run-02_T1w.nii.gz']


In [4]:
# Function to extract tag from file path
def extract_tag_from_file_path(file_path):
    file_path = file_path.replace(data_path[:-1], '')
    return file_path.split('\\')[2][-5:]

try:
    data_loader_df = pd.read_csv('data_loader_df.csv')
except:
    # Create a new DataFrame to store the results
    data_loader_df = pd.DataFrame(columns=['file_path', 'label'])

    # Iterate over the file list and find the corresponding CDR value
    for file_path in file_ls:
        tag = extract_tag_from_file_path(file_path)
        cdr_value = filtered_diagnostic_df.loc[filtered_diagnostic_df['file_tag'] == tag, 'cdr'].values
        if cdr_value.size > 0:  # Check if cdr_value is not empty
            new_row = pd.DataFrame({'file_path': [file_path], 'label': [cdr_value[0]]})
            data_loader_df = pd.concat([data_loader_df, new_row], ignore_index=True)
            data_loader_df.to_csv('data_loader_df.csv')

print(data_loader_df)

      Unnamed: 0                                          file_path  label
0              0  D:/DL/oasis-scripts/download_scans/bids\sub-OA...    0.0
1              1  D:/DL/oasis-scripts/download_scans/bids\sub-OA...    0.0
2              2  D:/DL/oasis-scripts/download_scans/bids\sub-OA...    0.0
3              3  D:/DL/oasis-scripts/download_scans/bids\sub-OA...    0.0
4              4  D:/DL/oasis-scripts/download_scans/bids\sub-OA...    0.0
...          ...                                                ...    ...
2991        2991  D:/DL/oasis-scripts/download_scans/bids\sub-OA...    0.0
2992        2992  D:/DL/oasis-scripts/download_scans/bids\sub-OA...    1.0
2993        2993  D:/DL/oasis-scripts/download_scans/bids\sub-OA...    0.0
2994        2994  D:/DL/oasis-scripts/download_scans/bids\sub-OA...    0.5
2995        2995  D:/DL/oasis-scripts/download_scans/bids\sub-OA...    0.0

[2996 rows x 3 columns]


This section is to extract 2D slices from 3D volumetric MRI scans. It iterates through the T1-weighted scans and slices between indexes 100-160, choosing a slice for every 3 slices. If the scan has less then 160 slices, then the middle slice will be taken of which there is only 1. Eventually there should be a dataset of 58,376. 

JPG is the saved file format due to its smaller size. NPY was tried but was 840GB and hence rejected.

**Run only once (roughly 7-20 mins depending on CPU).**

Due to an extremely imbalanced dataset (Label 0.0: 50216; Label 1.0: 5748; Label 2.0: 1971; Label 3.0: 441), weights will be used. 

In [5]:
from collections import defaultdict
import os
import pandas as pd
import torch

save_dir = data_path[:-5] + "preprocessed_images"
# Dictionary to store the count of each label
class_counts = defaultdict(int)

# List to store file paths and labels
file_paths_labels = []

# Iterate over all files in the save directory
for filename in os.listdir(save_dir):
    if filename.startswith("label_") and filename.endswith(".jpg"):
        # Extract the label from the filename and convert it to a decimal (float)
        label = float(filename.split("_")[1])
        # Increment the count for this label
        class_counts[label] += 1
        # Append the full file path and label to the list
        full_path = os.path.join(save_dir, filename)
        file_paths_labels.append((full_path, label))

dementia_count = 0
healthy_count = 0
# Print the count of each label
for label, count in class_counts.items():
    print(f"Label {label}: {count}")
    if label >0:
        dementia_count+=count
    else:
        healthy_count+=count
# Create a DataFrame from the list of file paths and labels
jpg_data_loader_df = pd.DataFrame(file_paths_labels, columns=['file_path', 'label'])

## Print the first few rows of the DataFrame
# print(jpg_data_loader_df.head())

# Calculate weights: Inverse of the frequency seems like a simple choice
total_count = sum(class_counts.values())
weights = {k: total_count / v for k, v in class_counts.items()}
print(weights)
# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# Convert weights to a tensor, ensuring the labels are in the correct order
weights_tensor = torch.tensor([weights[0], weights[1], weights[2], weights[3]], dtype=torch.float32)

print(weights_tensor)
weights_tensor = weights_tensor.to(device)
jpg_data_loader_df.to_csv('jpg_data_loader_df.csv')

all_dementia_weight = (healthy_count+dementia_count)/dementia_count
# all_dementia_weight = 3
print(all_dementia_weight)


Label 0.0: 50216
Label 1.0: 5748
Label 2.0: 1971
Label 3.0: 441
{0.0: 1.1624980086028358, 1.0: 10.155880306193458, 2.0: 29.617453069507864, 3.0: 132.3718820861678}
cuda
tensor([  1.1625,  10.1559,  29.6175, 132.3719])
7.153921568627451


## Functions

Functions for training the model. Includes NiftiDataset, HierarchicalCrossEntropyLoss, load_model, train_model and evaluate_model.

In [6]:
from torch.utils.data import DataLoader, random_split

class NiftiDataset(Dataset):
    def __init__(self, dataframe, preprocessed_dir):
        self.dataframe = dataframe
        self.preprocessed_dir = preprocessed_dir

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        label = row['label']
        img_path = row['file_path']  # Use the exact path from the dataframe
        scan = cv2.imread(img_path)
        # scan = cv2.resize(scan, (224,224), interpolation=cv2.INTER_LINEAR)
        scan = scan / 255.0  # Normalize the image to [0, 1]
        scan_tensor = torch.from_numpy(scan).float()
        
        # If the image is grayscale, we use unsqueeze to add the channel dimension
        if len(scan.shape) == 2:
            scan_tensor = scan_tensor.unsqueeze(0)  # Add channel dimension for grayscale image
        else:
            scan_tensor = scan_tensor.permute(2, 0, 1)  # Rearrange dimensions for color image
        
        label_tensor = torch.tensor(label).long()

        return scan_tensor, label_tensor

dataset = NiftiDataset(jpg_data_loader_df, preprocessed_dir=save_dir)

# Create a list of labels for stratified splitting
labels = jpg_data_loader_df.iloc[:, 1].values
myset = set(labels)
print(myset)

# Define the ratios for splitting
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

# Calculate the sizes for each split
total_size = len(dataset)
train_size = int(total_size * train_ratio)
val_size = int(total_size * val_ratio)
test_size = total_size - train_size - val_size

# Split the dataset into train, validation, and test sets
train_dataset, val_test_dataset = random_split(dataset, [train_size, val_size + test_size])
val_dataset, test_dataset = random_split(val_test_dataset, [val_size, test_size])

# Create dataloaders for each split
batch_size = 32  # Adjust the batch size as needed
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Function to count labels in a dataset
def count_labels(dataset):
    label_counts = {}
    for _, label_tensor in dataset:
        label = label_tensor.item()
        if label in label_counts:
            label_counts[label] += 1
        else:
            label_counts[label] = 1
    return label_counts


{0.0, 1.0, 2.0, 3.0}


### Ensemble learning

In [7]:
import os
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, precision_score, recall_score

def evaluate_ensemble(models, test_dataloader, device, method='averaging', weights=None):
    predictions = []
    all_labels = []

    if method in ['weighted_ensemble', 'weighted_max_voting']:
        if weights is None:
            # Initialize weights with equal values
            weights = [1.0 / len(models)] * len(models)
        elif len(weights) != len(models):
            raise ValueError("Number of weights must be equal to the number of models.")

    for inputs, labels in test_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        if method == 'averaging':
            ensemble_outputs = []
            for i, model in enumerate(models):
                model.cuda()
                model.eval()
                outputs = model(inputs)

                if hasattr(outputs, 'aux_logits'):
                    outputs = outputs.logits

                ensemble_outputs.append(outputs)

            ensemble_outputs = torch.stack(ensemble_outputs, dim=0)
            ensemble_mean = torch.mean(ensemble_outputs, dim=0)
            _, predicted = torch.max(ensemble_mean, dim=1)
        elif method == 'max_voting':
            ensemble_votes = torch.zeros((len(models), len(inputs)), dtype=torch.long).to(device)
            for i, model in enumerate(models):
                model.cuda()
                model.eval()
                outputs = model(inputs)

                if hasattr(outputs, 'aux_logits'):
                    outputs = outputs.logits

                _, predicted = torch.max(outputs, dim=1)
                ensemble_votes[i] = predicted

            predicted, _ = torch.mode(ensemble_votes, dim=0)
        elif method == 'weighted_averaging':
            ensemble_outputs = []
            for i, model in enumerate(models):
                model.cuda()
                model.eval()
                outputs = model(inputs)

                if hasattr(outputs, 'aux_logits'):
                    outputs = outputs.logits

                outputs *= weights[i]
                ensemble_outputs.append(outputs)

            ensemble_mean = torch.mean(torch.stack(ensemble_outputs, dim=0), dim=0)
            _, predicted = torch.max(ensemble_mean, dim=1)
            

        predictions.extend(predicted.tolist())
        all_labels.extend(labels.tolist())

    cm = confusion_matrix(all_labels, predictions)
    print(cm)
    class_accuracies = cm.diagonal() / cm.sum(axis=1)
    print(class_accuracies)
    
    accuracy = torch.sum(torch.tensor(predictions) == torch.tensor(all_labels)).item() / len(predictions)
    f1 = f1_score(all_labels, predictions, average='macro')
    precision = precision_score(all_labels, predictions, average='macro')
    recall = recall_score(all_labels, predictions, average='macro')
    
    print(f'Test Accuracy: {accuracy:.4f}, Test F1: {f1:.4f}, Test Precision: {precision:.4f}, Test Recall: {recall:.4f}')



#### Load models from checkpoints

In [8]:
resnet = models.resnet18()
num_features = resnet.fc.in_features
resnet.fc = nn.Linear(num_features, 4)

checkpoint = torch.load("checkpoints\\best_resnet18_w_crossentropy.pth", map_location=torch.device('cuda'))
resnet.load_state_dict(checkpoint)

<All keys matched successfully>

In [9]:
inception = models.inception_v3()
num_features = inception.fc.in_features
inception.fc = nn.Linear(num_features,4)

num_features_aux = inception.AuxLogits.fc.in_features
inception.AuxLogits.fc = nn.Linear(num_features_aux, 4)

checkpoint = torch.load("checkpoints\\best_inception_v3_w_crossentropy.pth", map_location=torch.device('cuda'))
inception.load_state_dict(checkpoint)

d:\DL\env\Lib\site-packages\torchvision\models\inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


<All keys matched successfully>

In [10]:
import timm
vit = timm.create_model('vit_small_patch16_224', pretrained=False)

# Modify the final layer to match the number of classes (4 in this case)
num_features = vit.head.in_features
vit.head = nn.Linear(num_features, 4)
checkpoint = torch.load("checkpoints\\best_vit_small_patch16_224_w_crossentropy.pth", map_location=torch.device('cuda'))
vit.load_state_dict(checkpoint)

d:\DL\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<All keys matched successfully>

#### Ignore

In [11]:
models = [resnet, inception]
device = torch.device("cuda")
weights = [0.6, 0.4]
evaluate_ensemble(models, test_dataloader, device, method='averaging')
evaluate_ensemble(models, test_dataloader, device, method='max_voting')
evaluate_ensemble(models, test_dataloader, device, method='weighted_averaging', weights=weights)

[[4862   76   55   11]
 [ 171  409    1    0]
 [  46    1  161    0]
 [   7    1    0   37]]
[0.9716227  0.70395869 0.77403846 0.82222222]
Test Accuracy: 0.9368, Test F1: 0.8207, Test Precision: 0.8271, Test Recall: 0.8180
[[4943   43   17    1]
 [ 445  136    0    0]
 [ 124   35   49    0]
 [  30    3    3    9]]
[0.98780975 0.23407917 0.23557692 0.2       ]
Test Accuracy: 0.8799, Test F1: 0.4898, Test Precision: 0.7822, Test Recall: 0.4144
[[4855   96   44    9]
 [ 147  433    1    0]
 [  33    1  174    0]
 [   6    1    0   38]]
[0.97022382 0.74526678 0.83653846 0.84444444]
Test Accuracy: 0.9421, Test F1: 0.8466, Test Precision: 0.8454, Test Recall: 0.8491


In [12]:
models = [resnet, vit]
device = torch.device("cuda")
weights = [0.6, 0.4]
evaluate_ensemble(models, test_dataloader, device, method='averaging')
evaluate_ensemble(models, test_dataloader, device, method='max_voting')
evaluate_ensemble(models, test_dataloader, device, method='weighted_averaging', weights=weights)

d:\DL\env\Lib\site-packages\timm\models\vision_transformer.py:86: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  x = F.scaled_dot_product_attention(


[[4800  134   61    9]
 [ 113  468    0    0]
 [  21    0  187    0]
 [   7    1    0   37]]
[0.95923261 0.80550775 0.89903846 0.82222222]
Test Accuracy: 0.9407, Test F1: 0.8473, Test Precision: 0.8265, Test Recall: 0.8715
[[4970   34    0    0]
 [ 513   68    0    0]
 [ 191   17    0    0]
 [  45    0    0    0]]
[0.99320544 0.11703959 0.         0.        ]
Test Accuracy: 0.8630, Test F1: 0.2803, Test Precision: 0.3601, Test Recall: 0.2776


d:\DL\env\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


[[4787  142   66    9]
 [ 107  474    0    0]
 [  20    0  187    1]
 [   6    1    0   38]]
[0.95663469 0.81583477 0.89903846 0.84444444]
Test Accuracy: 0.9397, Test F1: 0.8461, Test Precision: 0.8180, Test Recall: 0.8790


In [13]:
models = [vit, inception]
device = torch.device("cuda")
evaluate_ensemble(models, test_dataloader, device, method='averaging')
evaluate_ensemble(models, test_dataloader, device, method='max_voting')

[[4175  322  463   44]
 [ 458   61   57    5]
 [ 133   30   44    1]
 [  32    2    4    7]]
[0.83433253 0.10499139 0.21153846 0.15555556]
Test Accuracy: 0.7343, Test F1: 0.3063, Test Precision: 0.3044, Test Recall: 0.3266
[[4963   41    0    0]
 [ 575    6    0    0]
 [ 199    9    0    0]
 [  45    0    0    0]]
[0.99180655 0.01032702 0.         0.        ]
Test Accuracy: 0.8511, Test F1: 0.2348, Test Precision: 0.2414, Test Recall: 0.2505


d:\DL\env\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [14]:
models = [resnet, vit, inception]
device = torch.device("cuda")
weights = [0.44, 0.28, 0.28]
evaluate_ensemble(models, test_dataloader, device, method='averaging')
evaluate_ensemble(models, test_dataloader, device, method='max_voting')
evaluate_ensemble(models, test_dataloader, device, method='weighted_averaging', weights=weights)

[[4888   65   44    7]
 [ 183  397    1    0]
 [  52    1  155    0]
 [  13    0    0   32]]
[0.97681855 0.68330465 0.74519231 0.71111111]
Test Accuracy: 0.9373, Test F1: 0.8116, Test Precision: 0.8512, Test Recall: 0.7791
[[4921   67   15    1]
 [ 449  132    0    0]
 [ 159    4   45    0]
 [  36    0    0    9]]
[0.98341327 0.22719449 0.21634615 0.2       ]
Test Accuracy: 0.8748, Test F1: 0.4828, Test Precision: 0.7961, Test Recall: 0.4067
[[4870   85   41    8]
 [ 147  433    1    0]
 [  36    1  171    0]
 [   7    1    0   37]]
[0.97322142 0.74526678 0.82211538 0.82222222]
Test Accuracy: 0.9440, Test F1: 0.8472, Test Precision: 0.8550, Test Recall: 0.8407


In [15]:
models = [resnet, vit, inception]
device = torch.device("cuda")
weights = [0.44, 0.28, 0.28]
evaluate_ensemble(models, test_dataloader, device, method='averaging')
evaluate_ensemble(models, test_dataloader, device, method='max_voting')
evaluate_ensemble(models, test_dataloader, device, method='weighted_averaging', weights=weights)

[[4888   65   44    7]
 [ 183  397    1    0]
 [  52    1  155    0]
 [  13    0    0   32]]
[0.97681855 0.68330465 0.74519231 0.71111111]
Test Accuracy: 0.9373, Test F1: 0.8116, Test Precision: 0.8512, Test Recall: 0.7791
[[4921   67   15    1]
 [ 449  132    0    0]
 [ 159    4   45    0]
 [  36    0    0    9]]
[0.98341327 0.22719449 0.21634615 0.2       ]
Test Accuracy: 0.8748, Test F1: 0.4828, Test Precision: 0.7961, Test Recall: 0.4067
[[4870   85   41    8]
 [ 147  433    1    0]
 [  36    1  171    0]
 [   7    1    0   37]]
[0.97322142 0.74526678 0.82211538 0.82222222]
Test Accuracy: 0.9440, Test F1: 0.8472, Test Precision: 0.8550, Test Recall: 0.8407


###  Ensemble Learning

In [16]:
import torch
import torch.nn as nn
import torchvision.models as models

class ResNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResNetBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = None
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample is not None:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

import torch
import torch.nn as nn
import torchvision.models as models

class ResNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResNetBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = None
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample is not None:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out


class UNet_with_ResNet_Blocks(nn.Module):
    def __init__(self, num_classes):
        super(UNet_with_ResNet_Blocks, self).__init__()
        self.encoder1 = ResNetBlock(3, 64)
        self.pool = nn.MaxPool2d(2, 2)
        self.encoder2 = ResNetBlock(64, 128)
        self.bottleneck = ResNetBlock(128, 256)
        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.decoder2 = ResNetBlock(256, 128)
        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.decoder1 = ResNetBlock(128, 64)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(64, num_classes)

    def forward(self, x):
        # Encoder
        enc1 = self.encoder1(x)
        x = self.pool(enc1)
        enc2 = self.encoder2(x)
        x = self.pool(enc2)
        x = self.bottleneck(x)
        x = self.upconv2(x)
        x = torch.cat((x, enc2), dim=1)
        x = self.decoder2(x)
        x = self.upconv1(x)
        x = torch.cat((x, enc1), dim=1)
        x = self.decoder1(x)

        # Classification
        x = self.avg_pool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x
    
class AttentionLayer(nn.Module):
    def __init__(self, channel, reduction=16):
        super(AttentionLayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

class VGG_Attention(nn.Module):
    def __init__(self, num_classes=4):
        super(VGG_Attention, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            AttentionLayer(64),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            AttentionLayer(128),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            AttentionLayer(256),

            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            AttentionLayer(512),

            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            AttentionLayer(512),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
    
import torch
import torch.nn as nn
import torchvision.models as models

class ChannelAttention(nn.Module):
    def __init__(self, in_channels, reduction_ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc = nn.Sequential(
            nn.Conv2d(in_channels, in_channels // reduction_ratio, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(in_channels // reduction_ratio, in_channels, 1, bias=False)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        out = avg_out + max_out
        return self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size, padding=kernel_size // 2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        out = torch.cat([avg_out, max_out], dim=1)
        out = self.conv(out)
        return self.sigmoid(out)

class CBAM(nn.Module):
    def __init__(self, in_channels, reduction_ratio=16, kernel_size=7):
        super(CBAM, self).__init__()
        self.channel_attention = ChannelAttention(in_channels, reduction_ratio)
        self.spatial_attention = SpatialAttention(kernel_size)

    def forward(self, x):
        x = x * self.channel_attention(x)
        x = x * self.spatial_attention(x)
        return x

class CBAMResNet(nn.Module):
    def __init__(self, model=models.resnet50(), num_classes=4, in_channels_ls = [256, 512, 1024, 2048]):
        super(CBAMResNet, self).__init__()
        self.base_model = model
        self.cbam1 = CBAM(in_channels=in_channels_ls[0])
        self.cbam2 = CBAM(in_channels=in_channels_ls[1])
        self.cbam3 = CBAM(in_channels=in_channels_ls[2])
        self.cbam4 = CBAM(in_channels=in_channels_ls[3])

        num_features = self.base_model.fc.in_features
        self.base_model.fc = nn.Linear(num_features, num_classes)

    def forward(self, x):
        x = self.base_model.conv1(x)
        x = self.base_model.bn1(x)
        x = self.base_model.relu(x)
        x = self.base_model.maxpool(x)

        x = self.base_model.layer1(x)
        x = self.cbam1(x)

        x = self.base_model.layer2(x)
        x = self.cbam2(x)

        x = self.base_model.layer3(x)
        x = self.cbam3(x)

        x = self.base_model.layer4(x)
        x = self.cbam4(x)

        x = self.base_model.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.base_model.fc(x)

        return x
    
class SeparableConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        super(SeparableConv2d, self).__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size, stride, padding, groups=in_channels)
        self.pointwise = nn.Conv2d(in_channels, out_channels, 1, 1, 0)

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        return x

class XceptionInceptionModule(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(XceptionInceptionModule, self).__init__()
        self.branch1 = SeparableConv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.branch2 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels // 2, kernel_size=1),
            nn.ReLU(),
            SeparableConv2d(out_channels // 2, out_channels, kernel_size=3, padding=1)
        )
        # Removed one branch for simplification
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels, out_channels, kernel_size=1)
        )

    def forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch4 = self.branch4(x)
        return torch.cat([branch1, branch2, branch4], 1)

class XceptionInceptionNet(nn.Module):
    def __init__(self, num_classes=4):
        super(XceptionInceptionNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.xception_inception = XceptionInceptionModule(32, 64)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(64 * 3, num_classes)  # Adjusted for the reduced number of branches

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.xception_inception(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [17]:
vgg_attention = VGG_Attention(num_classes=4)
checkpoint = torch.load("checkpoints\\best_VGG_Attention.pth", map_location=torch.device('cuda'))
vgg_attention.load_state_dict(checkpoint)

cbam18 = CBAMResNet(num_classes=4, model=models.resnet18(), in_channels_ls = [64, 128, 256, 512])
checkpoint = torch.load("checkpoints\\best_CBAMResNet.pth", map_location=torch.device('cuda'))
cbam18.load_state_dict(checkpoint)

unet_resnet = UNet_with_ResNet_Blocks(num_classes=4)
checkpoint = torch.load("checkpoints\\best_UNet_with_ResNet_Blocks_w_crossentropy.pth", map_location=torch.device('cuda'))
unet_resnet.load_state_dict(checkpoint)

xception = XceptionInceptionNet(num_classes=4)
checkpoint = torch.load("checkpoints\\best_XceptionInceptionNet.pth", map_location=torch.device('cuda'))
xception.load_state_dict(checkpoint)

<All keys matched successfully>

In [18]:
models = [vgg_attention, cbam18]
device = torch.device("cuda")
weights = [0.23, 0,77]
evaluate_ensemble(models, test_dataloader, device, method='averaging')
evaluate_ensemble(models, test_dataloader, device, method='weighted_averaging', weights=weights)

[[4233  709   51   11]
 [ 113  465    3    0]
 [  33    4  171    0]
 [   1    1    0   43]]
[0.84592326 0.80034423 0.82211538 0.95555556]
Test Accuracy: 0.8414, Test F1: 0.7723, Test Precision: 0.7293, Test Recall: 0.8560
[[5004    0    0    0]
 [ 581    0    0    0]
 [ 208    0    0    0]
 [  45    0    0    0]]
[1. 0. 0. 0.]
Test Accuracy: 0.8571, Test F1: 0.2308, Test Precision: 0.2143, Test Recall: 0.2500


d:\DL\env\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [19]:
models = [vgg_attention, unet_resnet]
device = torch.device("cuda")
weights = [0.37, 0.63]
evaluate_ensemble(models, test_dataloader, device, method='averaging')
evaluate_ensemble(models, test_dataloader, device, method='weighted_averaging', weights=weights)

[[3916 1067   16    5]
 [ 350  229    2    0]
 [ 137   65    5    1]
 [  31    8    3    3]]
[0.78257394 0.39414802 0.02403846 0.06666667]
Test Accuracy: 0.7114, Test F1: 0.3046, Test Precision: 0.3940, Test Recall: 0.3169
[[3806 1152   37    9]
 [ 333  240    7    1]
 [ 126   66   14    2]
 [  22   10    6    7]]
[0.76059153 0.4130809  0.06730769 0.15555556]
Test Accuracy: 0.6966, Test F1: 0.3438, Test Precision: 0.4096, Test Recall: 0.3491


In [20]:
models = [xception, cbam18]
weights = [0.23, 0.77]
device = torch.device("cuda")
evaluate_ensemble(models, test_dataloader, device, method='averaging')
evaluate_ensemble(models, test_dataloader, device, method='weighted_averaging', weights=weights)

[[4222  708   60   14]
 [ 118  460    3    0]
 [  32    3  173    0]
 [   1    1    0   43]]
[0.84372502 0.79173838 0.83173077 0.95555556]
Test Accuracy: 0.8390, Test F1: 0.7619, Test Precision: 0.7113, Test Recall: 0.8557
[[4177  727   80   20]
 [ 110  465    5    1]
 [  26    4  178    0]
 [   1    1    0   43]]
[0.83473221 0.80034423 0.85576923 0.95555556]
Test Accuracy: 0.8330, Test F1: 0.7411, Test Precision: 0.6763, Test Recall: 0.8616


In [21]:
models = [unet_resnet, xception]
device = torch.device("cuda")
weights = [0.38, 0.62]
evaluate_ensemble(models, test_dataloader, device, method='averaging')
evaluate_ensemble(models, test_dataloader, device, method='weighted_averaging', weights=weights)

[[4125  838   36    5]
 [ 390  185    6    0]
 [ 143   50   14    1]
 [  34    4    4    3]]
[0.82434053 0.31841652 0.06730769 0.06666667]
Test Accuracy: 0.7412, Test F1: 0.3224, Test Precision: 0.4044, Test Recall: 0.3192
[[4278  704   20    2]
 [ 429  150    2    0]
 [ 165   37    6    0]
 [  35    4    3    3]]
[0.85491607 0.25817556 0.02884615 0.06666667]
Test Accuracy: 0.7600, Test F1: 0.3092, Test Precision: 0.4582, Test Recall: 0.3022


In [22]:
models = [vgg_attention, unet_resnet, cbam18]
device = torch.device("cuda")
weights = [0.159, 0.539, 0.302]
evaluate_ensemble(models, test_dataloader, device, method='averaging')
evaluate_ensemble(models, test_dataloader, device, method='max_voting')
evaluate_ensemble(models, test_dataloader, device, method='weighted_averaging', weights=weights)

[[4224  734   41    5]
 [ 124  455    2    0]
 [  38    3  167    0]
 [   3    2    0   40]]
[0.8441247  0.78313253 0.80288462 0.88888889]
Test Accuracy: 0.8369, Test F1: 0.7750, Test Precision: 0.7569, Test Recall: 0.8298
[[4575  418   11    0]
 [ 361  220    0    0]
 [ 176    2   30    0]
 [  32    1    0   12]]
[0.91426859 0.37865749 0.14423077 0.26666667]
Test Accuracy: 0.8285, Test F1: 0.4809, Test Precision: 0.7411, Test Recall: 0.4260
[[4190  770   40    4]
 [ 126  454    1    0]
 [  41    5  162    0]
 [   3    2    0   40]]
[0.83733014 0.78141136 0.77884615 0.88888889]
Test Accuracy: 0.8301, Test F1: 0.7708, Test Precision: 0.7592, Test Recall: 0.8216


In [23]:
models = [xception, unet_resnet, cbam18 ]
device = torch.device("cuda")
weights = [0.167, 0.560, 0.273]
evaluate_ensemble(models, test_dataloader, device, method='averaging')
evaluate_ensemble(models, test_dataloader, device, method='max_voting')
evaluate_ensemble(models, test_dataloader, device, method='weighted_averaging', weights=weights)

[[4235  718   46    5]
 [ 131  448    2    0]
 [  37    3  168    0]
 [   3    2    0   40]]
[0.84632294 0.77108434 0.80769231 0.88888889]
Test Accuracy: 0.8378, Test F1: 0.7732, Test Precision: 0.7526, Test Recall: 0.8285
[[4406  587   11    0]
 [ 349  232    0    0]
 [ 164   14   30    0]
 [  32    1    0   12]]
[0.8804956  0.39931153 0.14423077 0.26666667]
Test Accuracy: 0.8016, Test F1: 0.4688, Test Precision: 0.7250, Test Recall: 0.4227
[[4202  759   39    4]
 [ 135  445    1    0]
 [  42    5  161    0]
 [   4    2    0   39]]
[0.83972822 0.76592083 0.77403846 0.86666667]
Test Accuracy: 0.8303, Test F1: 0.7664, Test Precision: 0.7585, Test Recall: 0.8116
